In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')
sys.path.append(workdir_ + 'myPythonTools/CASutils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import utils as uti
import numerical_utils as nuti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL
import filter_utils as fu

sys.path.append(workdir_ + 'PyRegridding/Drivers/')
import RegridField as RgF

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime
import yaml
#from box import Box

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(GrU)


In [ ]:
%%time
##########################################################
# Generate regirdding objetcs to be usedf later on
#---------------------------------------------------------

RgOb_ne240_x_ne16 = RgF.Horz( Src='ne240pg3', Dst='ne16pg3' ) 

RgOb_ne16_x_ne240 = RgF.Horz(  Src='ne16pg3' , Dst='ne240pg3' , RegridMethod='BILINEAR' ) 

RgOb_ne240_x_llOxO = RgF.Horz( Src='ne240pg3', Dst='latlonOxO' ) 


In [ ]:


with open('configure_plots.yaml', 'r') as file:
    cfg = yaml.safe_load(file)


######################################################

x='ne240x2'
#x='oldCTL' #'waccmL135'
exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
ymdPat = cfg[x]['ymdPat']
hsPat = cfg[x]['hsPat']
print( exp, subd, Src, Hkey, Dst, useri )

year,month,day,hour = 1985,1,1,0
second = hour  * 3600
ymdPat=str( year ).zfill(4) + '-' + str(month).zfill(2) + '-' + str(day).zfill(2) + '-' + str(second).zfill(5)

ymdPat='2004-07-15-*'
A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat=hsPat , ymdPat=ymdPat,verbose=True, open_dataset=True )



In [ ]:
Topo=xr.open_dataset(A.X.topography_file)
topoO=Topo.PHIS.values / 9.8

In [ ]:
ogO=A.X.OMEGA.values

print(ogO.shape)

ogOx2=RgF.Horz(xfld_Src=ogO , Src='ne240pg3', Dst='ne16pg3' , RegridObj_In=  RgOb_ne240_x_ne16  ) 

ogOx2xO=RgF.Horz(xfld_Src=ogOx2 , Src='ne16pg3' , Dst='ne240pg3', RegridObj_In= RgOb_ne16_x_ne240  ) 

latO,lonO = GrU.latlon( grid='ne240pg3' )

print( ogO.shape )
print( ogOx2.shape )
print( ogOx2xO.shape )



In [ ]:
A.X.area
print( np.sum(A.X.area.values ) , 4.*np.pi )

In [ ]:
#A.X.area

print( np.sum( A.X.area.values*ogOx2xO[0,:] ) , 4.*np.pi )

#plt.plot( ogOx2xO[0,:] )

In [ ]:
dog = ogO - ogOx2xO

In [ ]:
topoll =RgF.Horz(xfld_Src=topoO , Src='ne240pg3', Dst='latlonOxO' , RegridObj_In=  RgOb_ne240_x_llOxO  ) 


In [ ]:
dogll =RgF.Horz(xfld_Src=dog , Src='ne240pg3', Dst='latlonOxO' , RegridObj_In=  RgOb_ne240_x_llOxO  ) 


In [ ]:
ogOll =RgF.Horz(xfld_Src=ogO , Src='ne240pg3', Dst='latlonOxO' , RegridObj_In=  RgOb_ne240_x_llOxO  ) 
ogOx2xOll =RgF.Horz(xfld_Src=ogOx2xO , Src='ne240pg3', Dst='latlonOxO' , RegridObj_In=  RgOb_ne240_x_llOxO  ) 


In [ ]:

dolev = np.linspace(-1,1,num=21)
olev = np.linspace(-10,10,num=21)
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()


k=70
co=ax[0].contourf( ogOll[0,k,:,:] , levels = 0.5*olev, cmap='bwr' )
ax[0].set_xlim((500,1000))
ax[0].set_ylim((100,400))
cb=plt.colorbar( co )
co=ax[1].contourf( ogOx2xOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
ax[1].set_xlim((500,1000))
ax[1].set_ylim((100,400))
cb=plt.colorbar( co )
co=ax[2].contourf( dogll[0,k,:,:] , levels = 2*dolev , cmap='bwr' )
ax[2].set_xlim((500,1000))
ax[2].set_ylim((100,400))
cb=plt.colorbar( co )
#plt.colorbar()

In [ ]:
toot = (A.X.time[0].values.item() ).strftime('%Y-%m-%d:%HZ')
print( toot )

In [ ]:
A.exp

In [ ]:
plev=A.X.lev.values
#print(plev)
zlev= -7.0 * np.log( plev / 1000. )

In [ ]:
klist=[90,85, 80,75, 68, 65, 57 , 50 ,38,30,20,10 ]
klist=[85, 80, 68, 57 , 38,11 ]

zlev[klist]

In [ ]:


dolev = np.linspace(-1,1,num=21)
olev = np.linspace(-10,10,num=21)
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(3, 4, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

t=0
toot = (A.X.time[t].values.item() ).strftime('%Y-%m-%d:%HZ')
klist=[90,84, 80,75, 68, 65, 57 , 50 ,38,30,20,10 ]
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf( scak*dogll[t,k,:,:] , levels = 2*dolev , cmap='bwr' )
    co2 = ax[p].contour( topoll , levels = [1,10,1000], colors='black' )
    #ax[p].set_xlim((500,1000))
    #ax[p].set_ylim((100,400))
    ax[p].set_xlim((200,1300))
    ax[p].set_ylim((100,700))
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    cb=plt.colorbar( co )
    p=p+1
fig.suptitle( f'{A.exp}, time={toot}' , fontsize=20)
#plt.colorbar()
FigName=f'Hpass-omega.png'
Pu.save_link(png_filename = FigName )


In [ ]:


dolev = np.linspace(-1,1,num=21)
olev = np.linspace(-10,10,num=21)
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(3, 4, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

t=0
toot = (A.X.time[t].values.item() ).strftime('%Y-%m-%d:%HZ')
klist=[90,85, 80,75, 70, 65, 60 , 50 ,40,30,20,10 ]
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    co=ax[p].contourf( scak*ogOll[t,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co2 = ax[p].contour( topoll , levels = [1,10,1000], colors='black' )
    #ax[p].set_xlim((500,1000))
    #ax[p].set_ylim((100,400))
    ax[p].set_xlim((200,1300))
    ax[p].set_ylim((100,700))
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    cb=plt.colorbar( co )
    p=p+1
fig.suptitle( f'{A.exp}, time={toot}' , fontsize=20)
#plt.colorbar()
FigName=f'Raw-omega.png'
Pu.save_link(png_filename = FigName )


In [ ]:
###

In [ ]:
###

In [ ]:
###
poo=1

In [ ]:
###
xoo=poo

In [ ]:

plt.plot(ogOll[0,k,210,:] )
plt.plot(ogOx2xOll[0,k,210,:] )
#plt.plot(dogll[0,k,200,:] )


In [ ]:
print( np.min(te))

# Get the indices of the minimum value
min_index = np.unravel_index(np.argmin(te), te.shape)
print( min_index )

In [ ]:
print( te[ min_index[0] , min_index[1] ])

In [ ]:
plt.plot ( te[ : , min_index[1] ] )

In [ ]:
fig = plt.figure(figsize=( 20, 10 ))
plt.tricontourf( A.X.lon, A.X.lat, te[123,:] )

In [ ]:
# Find all indices where values are below 220
below_220_indices = np.where(te < 222)

# Convert these to a list of (row, col) pairs
below_220_coords = list(zip(below_220_indices[0], below_220_indices[1]))



In [ ]:
plt.tricontourf( lonO, latO, ogO[0,:]-ogOx2xO[0,80,:] )

In [ ]:
plt.tricontourf( lonO, latO, ogO[0,:]-ogOx2xO[0,:] ,levels=21, cmap='bwr' )

In [ ]:
plt.plot( ogO[0,80,:])
plt.plot( ogOx2xO[0,80,:])

In [ ]:
%%time
RegridObj = RgF.Horz(Src='ne240pg3', Dst='latlonOxO' )


In [ ]:
print("poo poo ")

In [ ]:
from scipy.ndimage import gaussian_filter as GF
from matplotlib.ticker import FuncFormatter

# Conversion function: km to degrees longitude
def km_to_lon(x, km_per_deg=111):
    return x / km_per_deg
# Conversion function: m to degrees longitude
def m_to_lon(x, m_per_deg=68_000):
    return (x / m_per_deg)-180.

# Create a formatter function using the conversion function
def longitude_formatter(x, pos):
    return f'{m_to_lon(x):.1f}°'

# Define tick positions in degrees and convert to km
#tick_positions_deg = [60, 120, 180, 240, 300]
tick_positions_deg = [-120, -60, 0, 60, 120]
tick_positions_m = [(deg+180.)* 68_000 for deg in tick_positions_deg]  # Assuming 68 km per degree


pog44km  = np.sqrt(np.average( (og44km[:,j0:j1,:])**2,axis=1) )
pzeta1km = np.sqrt( np.average( (zeta1km[:,j0:j1,:])**2,axis=1) )
pzeta5km = np.sqrt( np.average( (zeta5km[:,j0:j1,:])**2,axis=1) )
pog1km   = np.sqrt( np.average( (og1km[:,j0:j1,:])**2,axis=1) )
pog10km  = np.sqrt( np.average((og10km[:,j0:j1,:])**2,axis=1) )
pog20km  = np.sqrt( np.average((og20km[:,j0:j1,:])**2,axis=1) )
pshflx   = np.average( (shflx[:,j0:j1,:]),axis=1)
pwp1km   = np.average( (wp1km[:,j0:j1,:]),axis=1)
pthp1km   = np.average( (thp1km[:,j0:j1,:]),axis=1)


xx=np.linspace(0,1.,num=nx)*( 2.*np.pi*6_371_000* np.cos( np.radians( -52. ) ))
#xx=np.linspace(-0.5,0.5,num=nx)*( 2.*np.pi*6_371_000* np.cos( np.radians( -52. ) ))
tt=np.linspace(0,1.,num=nt)*6.*3600.*nt
fig,ax=plt.subplots(1,2,squeeze=False,figsize=(14,6))
#plt.contour(pog44km  ,colors='black',levels=5 )
cn0=ax[0,0].contourf(  xx,tt, GF( pog20km,sigma=[2,5] ), cmap='inferno',levels=np.linspace(0.,.08,num=11) )#, linestyles='solid' )#[-4,-3,-2])#,colors='blue',levels=5 )
ln0=ax[0,0].plot( tt*5 , tt, color='black'  ,  label='5ms$^{-1}$'   )
ln0=ax[0,0].plot( tt*10 , tt, '--', color='black' ,  label='10ms$^{-1}$' )
ln0=ax[0,0].plot( tt*20 , tt, '-.', color='black' ,  label='20ms$^{-1}$' )
ax[0,0].set_xlim( 0,xx.max())
# Set the tick positions and formatter for the x-axis
ax[0, 0].set_xticks(tick_positions_m)
ax[0, 0].xaxis.set_major_formatter(FuncFormatter(longitude_formatter))
ax[0,0].legend()
ax[0,0].set_title( '$\omega (20km)$' )

cn1=ax[0,1].contourf(  xx,tt, GF( pzeta1km,sigma=[2,5] ) ,cmap='inferno',levels=15)# , linestyles='solid' )#[-4,-3,-2])#,colors='blue',levels=5 )
ln1=ax[0,1].plot( tt*5 , tt, color='black' )
ln1=ax[0,1].plot( tt*10 , tt, '--', color='black' )
ln1=ax[0,1].plot( tt*20 , tt, '-.', color='black' )
ax[0,1].set_xlim( 0,xx.max())
# Set the formatter for the x-axis
ax[0, 1].set_xticks(tick_positions_m)
ax[0, 1].xaxis.set_major_formatter(FuncFormatter(longitude_formatter))
ax[0,1].set_title( '$\zeta (5km)$' )


fig.colorbar(cn0, ax=ax[0, 0], orientation='vertical')
fig.colorbar(cn1, ax=ax[0, 1], orientation='vertical')
#plt.contour(pshflx   ,colors='orange')

print( np.max(xx)) 

In [ ]:
print( 6_371_000* np.cos( np.radians( -52. ) ))

In [ ]:

fig,ax=plt.subplots(1,2,squeeze=False,figsize=(14,6) )
ax[0,0].scatter( GF(pzeta5km,sigma=[2,25])[0:,1000:2000].flatten() , GF((pog20km),sigma=[2,5])[:,1000:2000].flatten() ,s=.1)
ax[0,0].set_xlabel( '$\zeta (5km) $')
ax[0,0].set_ylabel( '$\omega (20km) $')
ax[0,1].scatter( GF(pzeta5km,sigma=[2,25])[0:,1000:2000].flatten() , GF((pog10km),sigma=[2,5])[:,1000:2000].flatten() ,s=.1)
ax[0,1].set_xlabel( '$\zeta (5km) $')
ax[0,1].set_ylabel( '$\omega (10km) $')
#plt.plot( np.log(pog20km)[:,1000:2000].flatten())

In [ ]:
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt

# Example 2D array
data = np.random.rand(100, 100)

# Define 1D kernels for x and y directions
kernel_x = np.array([1, 2, 1]) / 4  # Normalize so the sum is 1
kernel_y = np.array([1, 4, 6, 4, 1]) / 16  # Normalize so the sum is 1

# Apply the kernel along the x-direction
smoothed_data_x = scipy.ndimage.convolve1d(data, kernel_x, axis=1)

# Apply the kernel along the y-direction
smoothed_data = scipy.ndimage.convolve1d(smoothed_data_x, kernel_y, axis=0)

# Plot the original and smoothed data
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(data, cmap='viridis')
ax[0].set_title('Original Data')
ax[1].imshow(smoothed_data, cmap='viridis')
ax[1].set_title('Smoothed Data with Separable Kernels')
plt.show()

In [ ]:
og44km_var=np.average( og44km**2 , axis=0)

In [ ]:
zeta5km_var=np.average( zeta5km**2 , axis=0)

In [ ]:
og44km_var.shape

In [ ]:
plt.contourf( np.sqrt(og44km_var) ,levels=21)

In [ ]:

LL.Maps( fields=[np.sqrt(og44km_var )],lons=[lon_x],lats=[lat_x] ,clevs=[21],cmaps=['gist_ncar'])

In [ ]:

LL.Maps( fields=[np.sqrt(zeta5km_var )],lons=[lon_x],lats=[lat_x] ,clevs=[np.linspace(0.,0.0001,num=21)],cmaps=['gist_ncar'])